# 🤗 Aplicaciones prácticas de Transformers con Hugging Face

En esta clase vamos a usar modelos preentrenados en español para resolver tareas reales de Procesamiento del Lenguaje Natural (PLN), sin necesidad de entrenar modelos desde cero.

Trabajaremos con la librería `transformers` de Hugging Face, que permite usar modelos de tipo BERT, GPT y similares en muy pocas líneas de código.

---


In [ ]:
# Instalación de Hugging Face Transformers (solo una vez)
!pip install -q transformers

## 1. Cargando el pipeline de Hugging Face

Hugging Face proporciona "pipelines" que encapsulan todo el proceso: tokenización, modelo y decodificación. Solo tenés que indicar qué tarea querés hacer.


In [ ]:
from transformers import pipeline

## 2. Análisis de sentimiento en español

Vamos a usar un modelo entrenado para identificar si una frase expresa un **sentimiento positivo o negativo**. Este modelo fue entrenado con tweets en español.

In [ ]:
sentiment = pipeline("sentiment-analysis", model="finiteautomata/beto-sentiment-analysis")

In [ ]:
frases = [
    "Este lugar está buenísimo, la atención de diez",
    "Una experiencia horrible, me quiero ir",
    "Zafa, pero esperaba más",
    "Recomiendo totalmente este producto",
    "Nunca más compro acá"
]

In [ ]:
for frase in frases:
    print(f"{frase} → {sentiment(frase)}")

Este lugar está buenísimo, la atención de diez → [{'label': 'POS', 'score': 0.9988070726394653}]
Una experiencia horrible, me quiero ir → [{'label': 'NEG', 'score': 0.9991808533668518}]
Zafa, pero esperaba más → [{'label': 'NEG', 'score': 0.9256047010421753}]
Recomiendo totalmente este producto → [{'label': 'POS', 'score': 0.9985804557800293}]
Nunca más compro acá → [{'label': 'NEG', 'score': 0.9992826581001282}]


## 3. Clasificación de texto por tema (*zero-shot*)

¿Querés clasificar frases por categorías sin entrenar un modelo? ¡Esto es posible gracias al aprendizaje **zero-shot**!

El modelo puede asociar un texto con una o más **etiquetas** sugeridas por vos, aunque nunca fue entrenado específicamente para esas clases.

In [ ]:
classifier = pipeline("zero-shot-classification", model="Recognai/bert-base-spanish-wwm-cased-xnli")

In [ ]:
texto = "Lionel Messi firmó contrato con el Inter Miami y debutará esta semana."

etiquetas = ["deportes", "economía", "política", "espectáculos"]

print(classifier(texto, candidate_labels=etiquetas))

{'sequence': 'Lionel Messi firmó contrato con el Inter Miami y debutará esta semana.', 'labels': ['deportes', 'espectáculos', 'política', 'economía'], 'scores': [0.516789436340332, 0.2758041322231293, 0.1318969577550888, 0.0755094587802887]}


## 4. Resumen automático de textos

Este pipeline toma un texto largo y genera un resumen breve en español. Ideal para noticias, informes o textos descriptivos.

Usamos un modelo BERT2BERT adaptado al español.

In [ ]:
summarizer = pipeline(
    "summarization",
    model="csebuetnlp/mT5_multilingual_XLSum",
    tokenizer="csebuetnlp/mT5_multilingual_XLSum"
)

In [ ]:
parrafo = """
El Ministerio de Salud confirmó hoy que se ha logrado una reducción sostenida de casos de dengue en las últimas semanas.
Las campañas de prevención, sumadas a la llegada del frío, habrían contribuido a esta baja. Sin embargo, se pide a la población mantener las precauciones.
"""

resumen = summarizer(parrafo, max_length=50, min_length=20, do_sample=False)
print(resumen[0]['summary_text'])


El número de casos de dengue en Estados Unidos alcanzó un nuevo récord.


In [ ]:
texto = """
La inflación en Argentina ha mostrado una leve desaceleración en el último mes, según el informe del INDEC.
Sin embargo, los analistas advierten que la tendencia aún no se revierte, y que podrían esperarse aumentos para el próximo trimestre.
"""

resumen = summarizer(texto, max_length=60, min_length=25, do_sample=False)
print(resumen[0]['summary_text'])


El Fondo Monetario Internacional (INDEC) dijo que la inflación en Argentina se ha recuperado de una desaceleración en el último mes.


## 5. Traducción automática (Español → Inglés)

También podemos usar modelos preentrenados para **traducir textos**. En este caso, usaremos uno especializado para traducir del español al inglés.


In [ ]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")

In [ ]:
texto = "La inteligencia artificial está cambiando el mundo."

print(translator(texto)[0]['translation_text'])


Artificial intelligence is changing the world.


## 6. Generación de texto en español (GPT)

Con un modelo tipo GPT entrenado en español, podemos **generar texto a partir de un inicio dado**. Ideal para escribir contenido creativo, continuar frases, etc.


In [ ]:
generator = pipeline("text-generation", model="PlanTL-GOB-ES/gpt2-base-bne")

In [ ]:
prompt = "Un dinosaurio está"

resultado = generator(prompt, max_length=50, num_return_sequences=1)
print(resultado[0]['generated_text'])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=51) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Un dinosaurio está hecho de un sistema de construcción en el que el corazón se divide en dos partes, una que puede... 


## 7. Reflexión y discusión

- ¿Cuál de estos pipelines te pareció más sorprendente o útil?
    El pipeline de clasificación de texto por tema (zero-shot) me parece muy interesante. El resumen automático también es muy útil para procesar rápidamente grandes cantidades de texto.
- ¿Creés que estas herramientas podrían usarse en un proyecto real? ¿En cuál?
    Estoy seguro que lo puedo aplicar en alguno de mis proyectos como podrian ser:

    *1)Traducción automática*: Localización de contenido, comunicación multilingüe básica, traducción rápida de documentos.
    
    *2)Generación de texto*: Creación de contenido creativo, asistencia en la redacción de correos electrónicos o artículos, generación de descripciones de productos (con revisión humana).
- ¿Notaste errores o sesgos? ¿Por qué creés que aparecen?
    Sí, se pueden notar posibles "errores" o comportamientos inesperados, especialmente en el resumen automático. Por ejemplo, en el resumen del párrafo sobre el dengue, el modelo generó un resumen sobre casos de dengue en Estados Unidos, lo cual no estaba en el texto original. En el resumen sobre la inflación, mencionó al FMI en lugar del INDEC.

    Estos errores o sesgos pueden aparecer por varias razones:

    **Datos de entrenamiento**: Los modelos aprenden de los datos con los que fueron entrenados. Si esos datos contienen sesgos (por ejemplo, ciertos estereotipos, o información desactualizada/incorrecta), el modelo puede reflejarlos. En el caso de la generación de texto o resumen, si el modelo fue entrenado con datos de diferentes fuentes o regiones, puede generar información que no se alinea exactamente con el texto de entrada si hay información contradictoria o si el modelo prioriza cierta información de su entrenamiento general.

    **Limitaciones del modelo**: Aunque son muy potentes, los modelos no "entienden" el texto de la misma manera que un humano. Operan basándose en patrones estadísticos y la probabilidad de secuencias de palabras. Esto puede llevar a resúmenes que no capturan el significado exacto o a traducciones que no son perfectamente idiomáticas.

    **Naturaleza de la tarea**: Tareas como el resumen son intrínsecamente complejas y subjetivas. Lo que es un "buen" resumen puede variar. Los modelos generan un resumen basado en lo que es estadísticamente probable que sea relevante, pero no siempre aciertan.

    **Configuración de los parámetros**: Parámetros como max_length o min_length en el resumen pueden influir en la calidad y coherencia del resultado.
    Es crucial recordar que, si bien estos modelos son herramientas poderosas, a menudo requieren supervisión humana, especialmente en aplicaciones donde la precisión y la ausencia de sesgos son críticas.



## 8. Actividad libre (opcional si hay tiempo)

Explorá uno de los pipelines y diseñá tu propio experimento:

- Probá frases con sarcasmo o jergas locales.
- Resumí un artículo de Wikipedia.
- Traducí algo complejo (tecnológico, poético, etc.).
- Completá una frase usando estilo formal o informal.

Al final compartimos los hallazgos más interesantes con el grupo 👀



# Análisis de sentimiento en español

  Pruebo frases con sarcasmo o jergas locales estilo Pepe Mujica.

In [ ]:
frases2 = [
    "Pa' serte franco, esto está de novela, che",
    "Mirá, esto es un desastre, ni al peor enemigo se lo recomiendo",
    "No está ni tan mal ni tan bien... como pa' zafar",
    "Una joyita, lo digo con el corazón en la mano",
    "Esto no tiene vuelta, fue un clavo tremendo",
    "Le ponen garra, se nota, eso vale más que mil estrellas",
    "Una decepción bárbara, esperaba otra cosa",
    "Está bueno, sencillo pero cumplidor, como tiene que ser",
    "Nunca más caigo en esta, me salió el tiro por la culata",
    "Si querés algo bueno de verdad, esto es por donde va la cosa"
]

In [ ]:
for frase in frases2:
    print(f"{frase} → {sentiment(frase)}")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Pa' serte franco, esto está de novela, che → [{'label': 'NEU', 'score': 0.8798360228538513}]
Mirá, esto es un desastre, ni al peor enemigo se lo recomiendo → [{'label': 'NEG', 'score': 0.9989790916442871}]
No está ni tan mal ni tan bien... como pa' zafar → [{'label': 'POS', 'score': 0.9973741769790649}]
Una joyita, lo digo con el corazón en la mano → [{'label': 'POS', 'score': 0.9930917024612427}]
Esto no tiene vuelta, fue un clavo tremendo → [{'label': 'NEG', 'score': 0.9992641806602478}]
Le ponen garra, se nota, eso vale más que mil estrellas → [{'label': 'NEG', 'score': 0.9731284976005554}]
Una decepción bárbara, esperaba otra cosa → [{'label': 'NEG', 'score': 0.998916745185852}]
Está bueno, sencillo pero cumplidor, como tiene que ser → [{'label': 'NEU', 'score': 0.9683693647384644}]
Nunca más caigo en esta, me salió el tiro por la culata → [{'label': 'NEG', 'score': 0.999354898929596}]
Si querés algo bueno de verdad, esto es por donde va la cosa → [{'label': 'NEU', 'score': 0.97533